In [11]:
# assumptions.py
# takes user input

print(" Welcome to ValuEdge – DCF Valuation Engine")
print("Please enter the following financial assumptions:")

# Basic inputs
company_name = input("Company Name: ")
starting_revenue = float(input("Starting Revenue (in millions): "))
revenue_growth_rate = float(input("Annual Revenue Growth Rate (%) [e.g. 8]: ")) / 100
fcf_margin = float(input("Free Cash Flow Margin (%) [e.g. 15]: ")) / 100
tax_rate = float(input("Tax Rate (%) [e.g. 30]: ")) / 100
wacc = float(input("Discount Rate (WACC) (%) [e.g. 10]: ")) / 100
terminal_growth_rate = float(input("Terminal Growth Rate (%) [e.g. 2.5]: ")) / 100
forecast_years = int(input("Forecast Period (in years, e.g. 5 or 10): "))

# Package into a dictionary for clean access later
assumptions = {
    "company_name": company_name,
    "starting_revenue": starting_revenue,
    "revenue_growth_rate": revenue_growth_rate,
    "fcf_margin": fcf_margin,
    "tax_rate": tax_rate,
    "wacc": wacc,
    "terminal_growth_rate": terminal_growth_rate,
    "forecast_years": forecast_years
}

print(f"\n Assumptions for {company_name} saved successfully!")


 Welcome to ValuEdge – DCF Valuation Engine
Please enter the following financial assumptions:
Company Name: Alpha Inc
Starting Revenue (in millions): 600
Annual Revenue Growth Rate (%) [e.g. 8]: 8
Free Cash Flow Margin (%) [e.g. 15]: 15
Tax Rate (%) [e.g. 30]: 30
Discount Rate (WACC) (%) [e.g. 10]: 10
Terminal Growth Rate (%) [e.g. 2.5]: 2.5
Forecast Period (in years, e.g. 5 or 10): 5

 Assumptions for Alpha Inc saved successfully!


In [12]:
# Name: dcf_model.py

import pandas as pd

class DCFModel:
    def __init__(self, assumptions: dict):
        """
        Initialize the DCF model with input assumptions.
        """
        self.company_name = assumptions["company_name"]
        self.starting_revenue = assumptions["starting_revenue"]
        self.revenue_growth_rate = assumptions["revenue_growth_rate"]
        self.fcf_margin = assumptions["fcf_margin"]
        self.tax_rate = assumptions["tax_rate"]
        self.wacc = assumptions["wacc"]
        self.terminal_growth_rate = assumptions["terminal_growth_rate"]
        self.forecast_years = assumptions["forecast_years"]

        # Placeholder for forecasted results
        self.forecast_df = pd.DataFrame()

    def generate_forecast(self):
        """
        Generate revenue, FCF, and discounted cash flow projections year by year.
        """
        years = list(range(1, self.forecast_years + 1))
        revenue = []
        fcf = []
        discount_factors = []
        present_values = []

        current_revenue = self.starting_revenue

        for year in years:
            current_revenue *= (1 + self.revenue_growth_rate)
            current_fcf = current_revenue * self.fcf_margin
            discount_factor = 1 / ((1 + self.wacc) ** year)
            present_value = current_fcf * discount_factor

            revenue.append(current_revenue)
            fcf.append(current_fcf)
            discount_factors.append(discount_factor)
            present_values.append(present_value)

        # Store everything in a DataFrame
        self.forecast_df = pd.DataFrame({
            "Year": years,
            "Revenue": revenue,
            "Free Cash Flow": fcf,
            "Discount Factor": discount_factors,
            "Present Value of FCF": present_values
        })

        return self.forecast_df

    def calculate_terminal_value(self):
        """
        Calculate the terminal value using the Gordon Growth Model.
        """
        final_year_fcf = self.forecast_df["Free Cash Flow"].iloc[-1]
        terminal_value = (final_year_fcf * (1 + self.terminal_growth_rate)) / (self.wacc - self.terminal_growth_rate)
        terminal_discount_factor = 1 / ((1 + self.wacc) ** self.forecast_years)
        present_value_terminal = terminal_value * terminal_discount_factor
        return terminal_value, present_value_terminal

    def calculate_total_valuation(self):
        """
        Compute total firm value = sum of discounted FCFs + discounted terminal value
        """
        npv_of_fcf = self.forecast_df["Present Value of FCF"].sum()
        _, present_value_terminal = self.calculate_terminal_value()
        total_value = npv_of_fcf + present_value_terminal
        return total_value

In [13]:
# --- Scenario Definitions and Runner ---
# Case Sscenario: BULL, BASE, BEAR

def run_scenario_summaries(base_assumptions: dict):
    """
    Runs DCF valuation for Base, Bull, and Bear cases using the same starting revenue.
    Returns a dictionary of scenario results (enterprise values).
    """
    scenarios = {
        "Base": {
            "revenue_growth_rate": 0.06,
            "fcf_margin": 0.15,
            "wacc": 0.09,
            "terminal_growth_rate": 0.025
        },
        "Bull": {
            "revenue_growth_rate": 0.10,
            "fcf_margin": 0.18,
            "wacc": 0.08,
            "terminal_growth_rate": 0.035
        },
        "Bear": {
            "revenue_growth_rate": 0.02,
            "fcf_margin": 0.12,
            "wacc": 0.105,
            "terminal_growth_rate": 0.015
        }
    }

    results = {}

    for case, case_overrides in scenarios.items():
        # Create new assumptions based on base and override scenario-specific ones
        scenario_input = base_assumptions.copy()
        scenario_input.update(case_overrides)

        model = DCFModel(scenario_input)
        model.generate_forecast()
        total_value = model.calculate_total_valuation()

        results[case] = {
            "valuation": total_value,
            "assumptions": case_overrides
        }

    return results


In [14]:
# name: excel_exporter.py
%pip install xlsxwriter
import xlsxwriter
import os

def export_to_excel_multi(scenario_data: dict, company_name: str, chart_path: str = None):
    """
    Export multiple scenario DCF outputs to a structured Excel dashboard.

    Parameters:
        scenario_data (dict): Dictionary with Base, Bull, Bear outputs.
        company_name (str): Name of the company.
        chart_path (str): Optional file path of valuation chart image.
    """
    output_dir = "outputs"
    os.makedirs(output_dir, exist_ok=True)
    file_path = f"{output_dir}/{company_name}_DCF_Scenario_Valuation.xlsx"

    workbook = xlsxwriter.Workbook(file_path, {'nan_inf_to_errors': True})

    # Define formats
    bold = workbook.add_format({'bold': True})
    currency = workbook.add_format({'num_format': '$#,##0.00'})
    percent = workbook.add_format({'num_format': '0.00%'})

    # === Sheet 1: Summary Table ===
    summary_sheet = workbook.add_worksheet("Valuation Summary")
    summary_headers = [
        "Scenario", "Revenue Growth Rate", "FCF Margin", "WACC", "Terminal Growth",
        "Terminal Value", "Discounted Terminal Value", "Total Enterprise Value"
    ]
    for col, header in enumerate(summary_headers):
        summary_sheet.write(0, col, header, bold)

    for row, (scenario, data) in enumerate(scenario_data.items(), start=1):
        assumptions = data["assumptions"]
        summary_sheet.write(row, 0, scenario)
        summary_sheet.write(row, 1, assumptions["revenue_growth_rate"], percent)
        summary_sheet.write(row, 2, assumptions["fcf_margin"], percent)
        summary_sheet.write(row, 3, assumptions["wacc"], percent)
        summary_sheet.write(row, 4, assumptions["terminal_growth_rate"], percent)
        summary_sheet.write(row, 5, data["terminal_value"], currency)
        summary_sheet.write(row, 6, data["discounted_terminal_value"], currency)
        summary_sheet.write(row, 7, data["valuation"], currency)

    # === Sheet 2–4: Forecast Tables ===
    for scenario, data in scenario_data.items():
        forecast_df = data["forecast_df"]
        sheet = workbook.add_worksheet(f"{scenario} Forecast")

        # Write headers
        for col_num, header in enumerate(forecast_df.columns):
            sheet.write(0, col_num, header, bold)

        # Write forecast data
        for row_num, row in enumerate(forecast_df.values, start=1):
            for col_num, value in enumerate(row):
                if isinstance(value, (int, float)) and col_num > 0:
                    sheet.write(row_num, col_num, value, currency)
                else:
                    sheet.write(row_num, col_num, value)

    # === Sheet 5: Chart Image ===
    if chart_path and os.path.exists(chart_path):
        chart_sheet = workbook.add_worksheet("Valuation Chart")
        chart_sheet.insert_image("B2", chart_path)

    workbook.close()
    print(f" Excel file created: {file_path}")


In [16]:
# app.py
%pip install streamlit
import streamlit as st
import matplotlib.pyplot as plt
import io

st.set_page_config(page_title="ValuEdge DCF App", layout="wide")
st.sidebar.title("📊 ValuEdge – DCF Assumptions")

company_name = st.sidebar.text_input("Company Name", "Example Inc.")
starting_revenue = st.sidebar.number_input("Starting Revenue ($)", value=100_000_000)
forecast_years = st.sidebar.slider("Forecast Years", min_value=3, max_value=10, value=5)
tax_rate = st.sidebar.slider("Tax Rate", min_value=0.0, max_value=0.5, value=0.25)
scenario = st.sidebar.selectbox("Scenario", ["Base", "Bull", "Bear"])

scenario_map = {
    "Base": {"revenue_growth_rate": 0.06, "fcf_margin": 0.15, "wacc": 0.09, "terminal_growth_rate": 0.025},
    "Bull": {"revenue_growth_rate": 0.10, "fcf_margin": 0.18, "wacc": 0.08, "terminal_growth_rate": 0.035},
    "Bear": {"revenue_growth_rate": 0.02, "fcf_margin": 0.12, "wacc": 0.105, "terminal_growth_rate": 0.015}
}
defaults = scenario_map[scenario]

growth_rate = st.sidebar.slider("Revenue Growth", 0.0, 0.20, defaults["revenue_growth_rate"])
fcf_margin = st.sidebar.slider("FCF Margin", 0.0, 0.40, defaults["fcf_margin"])
wacc = st.sidebar.slider("WACC", 0.05, 0.15, defaults["wacc"])
terminal_growth = st.sidebar.slider("Terminal Growth", 0.0, 0.05, defaults["terminal_growth_rate"])
run = st.sidebar.button("Run DCF Valuation")

if run:
    base_inputs = {
        "company_name": company_name,
        "starting_revenue": starting_revenue,
        "forecast_years": forecast_years,
        "tax_rate": tax_rate
    }
    scenario_inputs = {
        "Base": {"revenue_growth_rate": 0.06, "fcf_margin": 0.15, "wacc": 0.09, "terminal_growth_rate": 0.025},
        "Bull": {"revenue_growth_rate": 0.10, "fcf_margin": 0.18, "wacc": 0.08, "terminal_growth_rate": 0.035},
        "Bear": {"revenue_growth_rate": 0.02, "fcf_margin": 0.12, "wacc": 0.105, "terminal_growth_rate": 0.015}
    }
    scenario_outputs = {}
    for case, overrides in scenario_inputs.items():
        merged = {**base_inputs, **overrides}
        model = DCFModel(merged)
        df = model.generate_forecast()
        tv, d_tv = model.calculate_terminal_value()
        ev = model.calculate_total_valuation()
        scenario_outputs[case] = {
            "forecast_df": df,
            "terminal_value": tv,
            "discounted_terminal_value": d_tv,
            "valuation": ev,
            "assumptions": overrides
        }

    st.subheader("💰 Valuation Summary")
    summary_df = pd.DataFrame([
        {"Scenario": k, "Growth": v["assumptions"]["revenue_growth_rate"], "WACC": v["assumptions"]["wacc"],
         "Margin": v["assumptions"]["fcf_margin"], "Terminal": v["assumptions"]["terminal_growth_rate"],
         "EV": v["valuation"]} for k, v in scenario_outputs.items()
    ])
    st.dataframe(summary_df.style.format({"EV": "${:,.0f}", "Growth": "{:.2%}", "WACC": "{:.2%}"}))

    # Chart
    fig, ax = plt.subplots()
    names = summary_df["Scenario"]
    values = summary_df["EV"]
    bars = ax.bar(names, values, color=["#4CAF50", "#2196F3", "#f44336"])
    for bar in bars:
        y = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, y + 1e6, f"${y:,.0f}", ha='center')
    ax.set_title(f"{company_name} - Valuation Scenarios")
    ax.set_ylabel("Enterprise Value")
    st.pyplot(fig)

    chart_path = f"{company_name}_chart.png"
    fig.savefig(chart_path)

    export_to_excel_multi(scenario_outputs, company_name, chart_path)
    with open(f"outputs/{company_name}_DCF_Scenario_Valuation.xlsx", "rb") as f:
        excel_bytes = f.read()
    st.download_button("📥 Download Excel Report", excel_bytes,
                       file_name=f"{company_name}_DCF_Scenario_Valuation.xlsx",
                       mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")


2025-07-04 15:37:15.655 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-04 15:37:15.657 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-04 15:37:15.708 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-04 15:37:15.709 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-04 15:37:15.710 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-04 15:37:15.712 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-04 15:37:15.713 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn